In [29]:
# Please install OpenAI SDK first: `pip3 install openai`
# Also make an account at https://platform.deepseek.com/usage and like your credit card. It should cost $1-2 per 300 pages, so dont top up too much money.

from openai import OpenAI

client = OpenAI(
    api_key="<INSERT-YOUR-DEEPSEEK-API-KEY-HERE>",
    base_url="https://api.deepseek.com/beta",
)


def translate(text):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {
                "role": "system",
                "content": "该文字来自一本叫《历史的游荡者》的书，请将该文字翻译成英文。",
            },
            {"role": "user", "content": text},
        ],
        stream=False,
        max_tokens=8000,
    )
    return response.choices[0].message.content

In [2]:
import os

with open("data/Li Shi De You Dang Zhe - Li Shuo.txt", "r", encoding="utf-8") as file:
    text = file.read()

len(text)

248250

In [23]:
splits = text.split("\n" * 6)
new_splits = []
cur_split = ""
for split in splits:
    cur_split += split
    if len(cur_split) >= 800:
        new_splits.append(cur_split)
        cur_split = ""
else:
    new_splits.append(cur_split)

In [33]:
# [len(s) for s in new_splits]

In [ ]:
def stream_translation(text):
    stream = translate(text)
    full_response = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            content = chunk.choices[0].delta.content
            print(content, end="", flush=True)
            full_response += content
    print()  # Print a newline at the end
    return full_response

In [ ]:
from concurrent.futures import ProcessPoolExecutor


def translate_chunk(chunk):
    return stream_translation(chunk)


with ProcessPoolExecutor(max_workers=5) as executor:
    translations = list(executor.map(translate_chunk, new_splits))

print(translations)

In [41]:
## Process Translated Book
with open("data/Li Shi De You Dang Zhe - Li Shuo - English.md", "r") as file:
    text = file.read()

splits = text.split("\n")
new_splits = []
for split in splits:
    if not split.startswith("#") and len(split) > 100:
        split = "&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + split
    elif (
        (not split.startswith("#"))
        and (not split.startswith("**"))
        and len(split) <= 100
        and split.strip() != ""
    ):
        split = f"**{split}**"
    new_splits.append(split)

with open("data/Li Shi De You Dang Zhe - Li Shuo - English - 2.md", "w") as file:
    file.write("\n".join(new_splits))